In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset 
from torchvision import datasets,transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score,accuracy_score,f1_score,precision_score,roc_curve,auc 

%matplotlib inline

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=nn.Conv2d(3,16,3,1,1)
    self.bn1=nn.BatchNorm2d(16)
    self.relu1=nn.ReLU()
    self.pool=nn.MaxPool2d(2)
    self.conv2=nn.Conv2d(16,32,3,1,1)
    self.relu2=nn.ReLU()
    self.conv3=nn.Conv2d(32,64,3,1,1)
    self.bn3=nn.BatchNorm2d(64)
    self.relu3=nn.ReLU()

    self.fc=nn.Linear(64*32*32,2)

  def forward(self,x):
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.relu1(x)
    x=self.pool(x)

    x=self.conv2(x)
    x=self.relu2(x)

    x=self.conv3(x)
    x=self.bn3(x)
    x=self.relu3(x)
    
    x=x.view(-1,64*32*32)
    x=self.fc(x)
    return x

class imgs(Dataset):
    def __init__(self,data,target,transform=None):
        self.data=data
        self.target=target
        self.transform=transform
    def __len__(self):
        return len(self.target)
    def __getitem__(self, index):
        img = self.data[index]
        lbl = self.target[index]
        return img, lbl

In [ ]:
#load dataset
dataset=ImageFolder('./dataset',transform=ToTensor())
kfc = KFold(n_splits=3, random_state=1, shuffle=True)

#initialize params
x=[]
y=[]
train_accuracy=[]
test_accuracy=[]
precision=[]
recall=[]
f1=[]

#process data to fit cross validation
for i in dataset:
    img, label=i
    x.append(img.numpy())
    y.append(label)
x=np.array(x)
y=np.array(y)

fold=1 #current fold
#cross validation
kfc = KFold(n_splits=3, random_state=1, shuffle=True)
for train_index,test_index in kfc.split(x):
    xtrain=x[train_index]
    xtest=x[test_index]
    ytrain=y[train_index]
    ytest=y[test_index]
    #process the data to fit the model
    ytrain=torch.tensor(ytrain,dtype=torch.long)
    ytest=torch.tensor(ytest,dtype=torch.long)
    xtr=[]
    xts=[]
    for i in xtrain:
        i=torch.from_numpy(i)
        xtr.append(i)
    for i in xtest:
        i=torch.from_numpy(i)
        xts.append(i)
    trainset=imgs(xtr,ytrain)
    testset=imgs(xts,ytest)
    trainload=DataLoader(trainset,8,shuffle=True,num_workers=0,pin_memory=True)
    testload=DataLoader(testset,8,shuffle=True,num_workers=0,pin_memory=True)
    
    #run model and calculate accuracy
    model=Net().to('cuda')
    optimizer = optim.Adadelta(model.parameters(), lr=0.001)
    loss_function = nn.CrossEntropyLoss()
    fin_train_acc=0
    fin_test_acc=0
    for apple in range(1,21):
        model.train()
        pd=[]
        tg=[]
        traccu=0.0
        trloss=0.0
        for data,target in trainload:
            target=target.cuda()
            optimizer.zero_grad()
            output=model(data.cuda().float())
            loss=loss_function(output,target)
            loss.backward()
            optimizer.step()
            trloss+=loss.cuda().data*data.size(0)
            _,prediction=torch.max(output.data,1)
            traccu+=int(torch.sum(prediction==target.data))
        traccu=traccu/len(trainset)
        model.eval()
        tsaccu=0.0
        tsloss=0.0
        for data,target in testload:
            target=target.cuda()
            output=model(data.cuda().float())
            _,prediction=torch.max(output.data,1)
            tsaccu+=int(torch.sum(prediction==target.data))
            for k in prediction:
                pd.append(k.item())
            for l in target.data:
                tg.append(l.item())
        tsaccu=tsaccu/len(testset)
        fin_train_acc=traccu
        fin_test_acc=tsaccu
        if tsaccu>=1.0:
            break;
    #save result
    train_accuracy.append(fin_train_acc)
    test_accuracy.append(fin_test_acc)
    precision.append(precision_score(tg,pd))
    f1.append(f1_score(tg,pd))
    recall.append(recall_score(tg,pd))
    #plot result
    print("Fold "+str(fold)+": ")
    print("Train accuracy: "+str(fin_train_acc))
    print("Test accuracy: "+str(fin_test_acc))
    print("Confusion matrix: ")
    print(confusion_matrix(tg,pd))
    fpr, tpr, threshold = roc_curve(tg,pd)
    auc1 = auc(fpr, tpr)
    plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc1)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.show()
    print()
    fold+=1
#show result
print("CNN Classifier Result 20 epoch (mean of 3 folds)")
print("Accuracy: ",sum(test_accuracy)/len(test_accuracy))
print("Precision: ",sum(precision)/len(precision))
print("F1: ",sum(f1)/len(f1))
print("Recall: ",sum(recall)/len(recall))